In [1]:
import matplotlib.pyplot as pp
import pycbc.noise
import pycbc.psd
import pycbc.filter
from pycbc.filter import matched_filter
from pycbc.waveform import get_td_waveform
from pycbc.waveform import get_fd_waveform
import numpy as np
from pycbc.vetoes import power_chisq
from pycbc.events.ranking import newsnr
import pandas as pd
from pycbc.filter import sigma
import h5py
from pycbc.types import timeseries
from pycbc.types import TimeSeries

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
def psd_data(conditioned):

    from pycbc.psd import interpolate, inverse_spectrum_truncation
    # Estimate the power spectral density

    # We use 4 second samples of our time series in Welch method.
    psd = conditioned.psd(4)

    # Now that we have the psd we need to interpolate it to match our data
    # and then limit the filter length of 1 / PSD. After this, we can
    # directly use this PSD to filter the data in a controlled manner

    psd = interpolate(psd, conditioned.delta_f)

    # 1/PSD will now act as a filter with an effective length of 4 seconds
    # Since the data has been highpassed above 30 Hz, and will have low values
    # below this we need to informat the function to not include frequencies
    # below this frequency. 
    psd = inverse_spectrum_truncation(psd, int(4 * conditioned.sample_rate),
                                      low_frequency_cutoff=30)

#     pp.loglog(psd.sample_frequencies, psd)
#     pp.ylabel('$Strain^2 / Hz$')
#     pp.xlabel('Frequency (Hz)')
#     pp.xlim(30, 1024)
    
    return psd

In [3]:
def gen_SNR(template,conditioned,psd):
    

    snr = matched_filter(template, conditioned,
                         psd=psd, low_frequency_cutoff=30)



    # The `matched_filter` function actually returns a 'complex' SNR.
    # What that means is that the real portion correponds to the SNR
    # associated with directly filtering the template with the data.
    # The imaginary portion corresponds to filtering with a template that
    # is 90 degrees out of phase. Since the phase of a signal may be 
    # anything, we choose to maximize over the phase of the signal.

   
    peak = abs(snr).numpy().argmax()
    snrp = abs(snr[peak])
    time = snr.sample_times[peak]
   
    
    return snr,time,peak,snrp

In [4]:
def gen_chisquare(temp,data,m_1,m_2,s_1,s_2,psd):
    
    n =int(0.72*pycbc.pnutils.get_freq('fSEOBNRv4Peak',m_1,m_2,s_1,s_2)**0.7)
    
    chisq = power_chisq(temp, data, n, psd=psd, low_frequency_cutoff=30.0)

    # convert to a reduced chisq
    chisq /= (n * 2) - 2
    
    
    
    return chisq


In [5]:
def snr_chisq_nsnr(num_templates,data_templates,psd,data,s):

    template_info = []


    for i in range(num_templates):
        T1 = data_templates[i]

        S1,time_peak,index_peak_bns,old_snr_bns = gen_SNR(T1,data,psd)
        S1.save(f"Signals_BBH_BNS/snr_complete_series/snr{i}.hdf")
        
        chisq_1 = gen_chisquare(T1,data,temp_bank[i][0],temp_bank[i][1],s,s,psd)
        chisq_val_bns = chisq_1[index_peak_bns]

        new_snr_bns = newsnr(abs(S1),chisq_1)
        
        

        snrp_bns = new_snr_bns[index_peak_bns]

        template_info.append([temp_bank[i][0],temp_bank[i][1],time_peak,old_snr_bns,snrp_bns,chisq_val_bns,i])

    return template_info
    


In [6]:
def recovery(data,n,data_templates,psd): 
    
  

    full__info =  snr_chisq_nsnr(n,data_templates,psd,data,0.8)
   
    
    

    info = {}
    for i in  full__info:
        info.update({i[4] : [i[0],i[1],i[2],i[3],i[5],i[6]]})


    snrp_1 = max(info.keys())
    time_1 = info.get(max(info.keys()))[2]
    m1_1 = info.get(max(info.keys()))[0]
    m2_1 = info.get(max(info.keys()))[1]
    chi_sq_1 = info.get(max(info.keys()))[4]
    index = info.get(max(info.keys()))[5]

    print("The signal detected is due to the following masses ",m1_1,"M and ", m2_1, "M at time ", time_1, "seconds", " with new SNR value as", snrp_1, " and chi~square value as ", chi_sq_1,"at index",index)
    
    
    return m1_1,m2_1,time_1,snrp_1,full__info,index



In [7]:
data_templates = []

for i in range(20):
    T = timeseries.load_timeseries(f"Signals_BBH_BNS/signal_bbh{i}.hdf")
    data_templates.append(T)
for i in range(20):
    R = timeseries.load_timeseries(f"Signals_BBH_BNS/signal_bns{i}.hdf")
    data_templates.append(R)
    
temp_bank = pd.read_csv('Signals_BBH_BNS/Load_bbh_bns.csv')
temp_bank = temp_bank.values.tolist()


In [8]:
for j in range(1):
    
    print("Recovery number:",j)
    print(' ')
    
    ts1 = timeseries.load_timeseries(f"Signals_BBH_BNS/Data_{j}.hdf")
    psd1 = psd_data(ts1)  
    mass_1,mass_2,time,snrp,total,index = recovery(ts1,40,data_templates,psd1)
    
    print(' ')


Recovery number: 0
 
The signal detected is due to the following masses  20.958661412898483 M and  32.86074720470542 M at time  255.800048828125 seconds  with new SNR value as 58.1806167251861  and chi~square value as  1.1003497315417372 at index 2
 


In [9]:
def find_bns(index):

    T_B1 = data_templates[index]
    T_B1_MF = timeseries.load_timeseries(f"Signals_BBH_BNS/snr_complete_series/snr{index}.hdf")

    A = []
    B = []
    maxim_SNR = []

    for i in range(40):
        if i != index:
            cross = matched_filter(data_templates[i], T_B1,
                             psd=psd1, low_frequency_cutoff=30)

            T_N1_MF = timeseries.load_timeseries(f"Signals_BBH_BNS/snr_complete_series/snr{i}.hdf")

            A_B1 = (T_B1_MF - (cross*T_N1_MF))/(1-(cross**2))
            A.append( A_B1 )

            A_N1 = (T_N1_MF - (cross*T_B1_MF))/(1-(cross**2))
            B.append(A_N1) 

            norm = (A_B1**2) + (A_N1**2) + 2*(A_B1*A_N1*cross)

            SNR = ((A_B1*T_B1_MF) + ( A_N1 * T_N1_MF))/norm

            peak_SNR = abs(SNR).numpy().argmax()

            SNR_p = abs(SNR[peak_SNR])

            maxim_SNR.append([i,SNR_p])
            
    M_SNR = 0
    idx = 0
    for i in range(len(maxim_SNR)):
        if M_SNR < maxim_SNR[i][1]:
            M_SNR = maxim_SNR[i][1]
            idx = maxim_SNR[i][0]
    return idx,M_SNR



In [10]:
def find_bbh(index):

    T_N1 = data_templates[index]
    T_N1_MF = timeseries.load_timeseries(f"Signals_BBH_BNS/snr_complete_series/snr{index}.hdf")


    A = []
    B = []
    maxim_SNR = []

    for i in range(40):
        if i != index:
            cross = matched_filter(data_templates[i], T_N1,
                             psd=psd1, low_frequency_cutoff=30)

            T_B1_MF = timeseries.load_timeseries(f"Signals_BBH_BNS/snr_complete_series/snr{i}.hdf")

            A_B1 = (T_B1_MF - (cross*T_N1_MF))/(1-(cross**2))
            A.append( A_B1 )

            A_N1 = (T_N1_MF - (cross*T_B1_MF))/(1-(cross**2))
            B.append(A_N1) 

            norm = (A_B1**2) + (A_N1**2) + 2*(A_B1*A_N1*cross)

            SNR = ((A_B1*T_B1_MF) + ( A_N1 * T_N1_MF))/norm

            peak_SNR = abs(SNR).numpy().argmax()

            SNR_p = abs(SNR[peak_SNR])

            maxim_SNR.append([i,SNR_p])
            
           
    M_SNR = 0
    idx = 0
    for i in range(len(maxim_SNR)):
        if M_SNR < maxim_SNR[i][1]:
            M_SNR = maxim_SNR[i][1]
            idx = maxim_SNR[i][0]
    return idx,M_SNR



In [11]:
index = 2
for i in range(10):
    xx,xy= find_bns(index)
    print(xx)
    aa,bb= find_bbh(xx)
    print(aa)
    index = aa
    

33
9
8
18
11
13
15
30
12
30
12
30
12
30
12
30
12
30
12
30
